In [1]:
import sys
sys.path.append("../")
import geopandas as gpd
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
import streamlit as st
import pandas as pd
import sys
from src.graph_layering.data_processing import Normalizer
from src.lightning.hetero_gnn_module import HeteroGNNModule
import os
import pandas as pd
import torch
import geopandas as gpd
import numpy as np
from sklearn.metrics import f1_score
from streamlit_folium import st_folium
from src.organized_datasets_creation.utils.nominatim import (
    convert_nominatim_name_to_filename,
    resolve_nominatim_city_name,
)
from shapely.geometry import Point
import joblib
from sklearn.preprocessing import StandardScaler


In [2]:
import os
os.path.exists("../data/")

True

In [6]:
# ACCIDENTS_LOCATION = "../data/downstream_tasks/accidents_prediction/accidents.csv"
GRAPH_DATA_DICT_PATH = "../data/results_showcase/zabka_shops/tabular_data_zabka.pkl"
MODEL_PATH = "../data/results_showcase/zabka_shops/model_zabka.pkl"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = joblib.load(GRAPH_DATA_DICT_PATH)
    model = joblib.load(MODEL_PATH)
    return data, model

In [7]:
data, model = load_graph_data_and_model()

In [8]:
data['Wrocław']['X'].head()

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,amenity_e-scooter_parking,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data.keys()

dict_keys(['Kraków', 'Poznań', 'Szczecin', 'Wrocław', 'Warszawa'])

In [10]:
'hex_id' in data['Wrocław']['X'].columns

False

dodanie kolumny `hex_id`

In [8]:
for City, city in [('Kraków', 'krakow'), ('Poznań', 'poznan'), ('Szczecin', 'szczecin'), ('Wrocław', 'wroclaw'), ('Warszawa', 'warszawa')]:
    df_region_id = pd.read_parquet(f"{ORGANIZED_HEXES_LOCATION}/{city}/2022/h9/count-embedder/dataset.parquet")['region_id']
    data[City]['X'] = data[City]['X'].join(df_region_id).rename(columns={'region_id': 'hex_id'})

In [9]:
'hex_id' in data['Wrocław']['X'].columns

True

In [10]:
import pickle

GRAPH_DATA_DICT_PATH = f'{GRAPH_DATA_DICT_PATH[:-4]}_hex_id.pkl'

with open(GRAPH_DATA_DICT_PATH, 'wb') as file:
    pickle.dump(data, file)


_____

sprawdzenie

In [3]:
# ACCIDENTS_LOCATION = "../data/downstream_tasks/accidents_prediction/accidents.csv"
GRAPH_DATA_DICT_PATH = "../data/results_showcase/zabka_shops/tabular_data_zabka_hex_id.pkl" # już podmieniona ścieżka 
MODEL_PATH = "../data/results_showcase/zabka_shops/model_zabka.pkl"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = joblib.load(GRAPH_DATA_DICT_PATH)
    model = joblib.load(MODEL_PATH)
    return data, model

In [4]:
data_with_hex_id, model = load_graph_data_and_model()
'hex_id' in data_with_hex_id['Wrocław']['X'].columns

True

In [11]:
data_with_hex_id['Wrocław']['X']

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group,hex_id
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,891e204e4a7ffff
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,891e204239bffff
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,891e2041837ffff
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,891e2047223ffff
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e2046657ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e2041137ffff
3164,0,3,0,0,0,0,0,0,0,21,...,0,0,0,0,0,0,0,0,0,891e2042273ffff
3165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e20430d7ffff


In [12]:
city_value = 'Wrocław'

In [13]:
for City in ['Kraków', 'Poznań', 'Szczecin', 'Wrocław', 'Warszawa']:
    data[City]['X'] = data_with_hex_id[City]['X'].drop(columns=['hex_id'])

In [15]:
data["Wrocław"]['X']

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,amenity_e-scooter_parking,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3164,0,3,0,0,0,0,0,0,0,21,...,0,0,0,0,0,0,0,0,0,0
3165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
folds = [
    ("Wrocław", "Kraków"),
    ("Kraków", "Poznań"),
    ("Poznań", "Szczecin"),
    ("Szczecin", "Warszawa"),
    ("Warszawa", "Wrocław"),
]
scaler = StandardScaler()
X = pd.concat(
[
    m["X"]
    for key, m in data.items()
    if key != city_value
]
).to_numpy()
y = (
pd.concat(
    [
        m["y"]
        for key, m in data.items()
        if key != city_value
    ]
)
.to_numpy()
.ravel()
)

X = scaler.fit_transform(X)
test_X = data[city_value]["X"].to_numpy()
test_X = scaler.transform(test_X)
test_y = data[city_value]["y"].to_numpy().ravel()
y_pred = model.predict(test_X)
y_proba = model.predict_proba(test_X)[:, 1]

auc = roc_auc_score(test_y, y_proba, average="micro")
accuracy = (y_pred == test_y).mean()
f1 = f1_score(
    test_y,
    y_pred,
    pos_label=1,
    average="binary",
)
f1

0.7599451303155007

In [12]:
GRAPH_DATA_DICT_PATH = "../data/results_showcase/accidents/data.pkl"
MODEL_PATH = "../data/results_showcase/accidents/model.ckpt"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = pd.read_pickle(GRAPH_DATA_DICT_PATH)

    model = HeteroGNNModule.load_from_checkpoint(
        MODEL_PATH, hetero_data=list(data.values())[3], map_location=torch.device("cpu")
    )
    return data, model


data1, model1 = load_graph_data_and_model()
data1

{'Wrocław, Poland': CityHeteroData(
   hex={
     x=[3168, 0],
     y=[3168],
   },
   osmnx_node={ x=[71641, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 9206] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 124386],
     edge_attr=[124386, 44],
   },
   (osmnx_node, connected_to, hex)={
     edge_index=[2, 71641],
     edge_attr=[71641, 0],
   }
 ),
 'Szczecin, Poland': CityHeteroData(
   hex={
     x=[3534, 0],
     y=[3534],
   },
   osmnx_node={ x=[64894, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 10163] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 106588],
     edge_attr=[106588, 44],
   },
   (osmnx_node, connected_to, hex)={
     edge_index=[2, 64894],
     edge_attr=[64894, 0],
   }
 ),
 'Poznań, Poland': CityHeteroData(
   hex={
     x=[2945, 0],
     y=[2945],
   },
   osmnx_node={ x=[60082, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 8486] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 9865

In [24]:
from shapely import wkt
zabka_loc = gpd.GeoDataFrame(columns=['city_name', 'geometry'])

path = "../data/downstream_tasks/zabka_shops/" 
for City, city in [('Kraków', 'krakow'), ('Poznań', 'poznan'), ('Szczecin', 'szczecin'), ('Wrocław', 'wroclaw'), ('Warszawa', 'warszawa')]:
    gdf = pd.read_csv(f"{path}zabka_data_{city}.csv")
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    temp_gdf = gpd.GeoDataFrame(gdf['geometry'], geometry='geometry')
    temp_gdf['city_name'] = City
    # data[City]['X'] = data[City]['X'].join(df_region_id).rename(columns={'region_id': 'hex_id'})
    display(temp_gdf)
    zabka_loc = pd.concat([zabka_loc, temp_gdf], ignore_index=True)

,geometry,city_name
0,POINT (19.91426 50.06739),Kraków
1,POINT (19.90577 50.08373),Kraków
2,POINT (20.01262 50.06982),Kraków
3,POINT (20.00939 50.07318),Kraków
4,POINT (19.98519 50.04042),Kraków
...,...,...
408,POINT (19.96670 50.10149),Kraków
409,POINT (20.01210 50.01679),Kraków
410,POINT (19.99996 50.01467),Kraków
411,POINT (19.93790 50.03325),Kraków


,geometry,city_name
0,POINT (16.90667 52.46039),Poznań
1,POINT (16.92348 52.39547),Poznań
2,POINT (16.88162 52.40058),Poznań
3,POINT (16.91494 52.38538),Poznań
4,POINT (16.93290 52.40930),Poznań
...,...,...
326,POINT (16.99463 52.36594),Poznań
327,POINT (16.92870 52.40379),Poznań
328,POINT (16.92511 52.40255),Poznań
329,POINT (16.90735 52.37150),Poznań


,geometry,city_name
0,POINT (14.56836 53.43506),Szczecin
1,POINT (14.50498 53.43167),Szczecin
2,POINT (14.50010 53.45490),Szczecin
3,POINT (14.65655 53.37843),Szczecin
4,POINT (14.53567 53.46701),Szczecin
...,...,...
188,POINT (14.64915 53.36450),Szczecin
189,POINT (14.54332 53.44647),Szczecin
190,POINT (14.57232 53.43501),Szczecin
191,POINT (14.49096 53.39488),Szczecin


,geometry,city_name
0,POINT (16.94955 51.10132),Wrocław
1,POINT (17.02829 51.06116),Wrocław
2,POINT (17.07466 51.13782),Wrocław
3,POINT (17.00675 51.08269),Wrocław
4,POINT (16.93386 51.09495),Wrocław
...,...,...
404,POINT (16.99051 51.12353),Wrocław
405,POINT (17.03028 51.11007),Wrocław
406,POINT (17.14071 51.10802),Wrocław
407,POINT (17.11924 51.10957),Wrocław


,geometry,city_name
0,POINT (21.17291 52.20562),Warszawa
1,POINT (20.98987 52.23572),Warszawa
2,POINT (21.02927 52.15541),Warszawa
3,POINT (21.01264 52.20832),Warszawa
4,POINT (20.92704 52.28513),Warszawa
...,...,...
932,POINT (21.05551 52.29839),Warszawa
933,POINT (21.05811 52.30265),Warszawa
934,POINT (20.99362 52.18255),Warszawa
935,POINT (21.04137 52.25028),Warszawa


In [25]:
zabka_loc

,city_name,geometry
0,Kraków,POINT (19.91426 50.06739)
1,Kraków,POINT (19.90577 50.08373)
2,Kraków,POINT (20.01262 50.06982)
3,Kraków,POINT (20.00939 50.07318)
4,Kraków,POINT (19.98519 50.04042)
...,...,...
2278,Warszawa,POINT (21.05551 52.29839)
2279,Warszawa,POINT (21.05811 52.30265)
2280,Warszawa,POINT (20.99362 52.18255)
2281,Warszawa,POINT (21.04137 52.25028)


In [26]:
zabka_loc.to_csv("../data/results_showcase/zabka_shops/zabka_locations.csv")

In [2]:
import pandas as pd
import geopandas as gpd

zabkas = pd.read_csv("../data/results_showcase/zabka_shops/zabka_locations.csv")
zabkas

,Unnamed: 0,city_name,geometry
0,0,Kraków,POINT (19.914264 50.067393)
1,1,Kraków,POINT (19.905769 50.083725)
2,2,Kraków,POINT (20.012619 50.069817)
3,3,Kraków,POINT (20.009387 50.073177)
4,4,Kraków,POINT (19.985185 50.040418)
...,...,...,...
2278,2278,Warszawa,POINT (21.055509 52.298391)
2279,2279,Warszawa,POINT (21.058111 52.302648)
2280,2280,Warszawa,POINT (20.993621 52.182553)
2281,2281,Warszawa,POINT (21.041369 52.250283)


In [3]:
zabkas = zabkas.drop("Unnamed: 0", axis=1)
zabkas

,city_name,geometry
0,Kraków,POINT (19.914264 50.067393)
1,Kraków,POINT (19.905769 50.083725)
2,Kraków,POINT (20.012619 50.069817)
3,Kraków,POINT (20.009387 50.073177)
4,Kraków,POINT (19.985185 50.040418)
...,...,...
2278,Warszawa,POINT (21.055509 52.298391)
2279,Warszawa,POINT (21.058111 52.302648)
2280,Warszawa,POINT (20.993621 52.182553)
2281,Warszawa,POINT (21.041369 52.250283)


In [22]:
from srai.plotting import plot_regions
from srai.regionalizers import geocode_to_region_gdf
from srai.regionalizers import H3Regionalizer
from srai.joiners import IntersectionJoiner

def zabka_per_region(city, gdf, resolution=9):
    area = geocode_to_region_gdf(f"{city}, Poland") 
    regionalizer = H3Regionalizer(resolution=resolution)
    regions = regionalizer.transform(area)
    joiner = IntersectionJoiner()
    zabka_per_region_df = (
        joiner.transform(regions, gdf)
        .reset_index()
        # .set_index("region_id")
        .groupby("region_id")
        .count()
    )

    zabka_per_region_df.reset_index(inplace=True)
    zabka_per_region_df.rename(columns={'feature_id': 'count_zabka'}, inplace=True)
    zabka_per_region_df = pd.merge(regions, zabka_per_region_df, how='left', on='region_id')
    zabka_per_region_df['count_zabka'].fillna(0, inplace=True)
    zabka_per_region_df['count_zabka'].astype(int)
    zabka_per_region_df = zabka_per_region_df.drop('geometry', axis=1)
    zabka_per_region_df = zabka_per_region_df.rename(columns={"region_id": "h3_id"})
    display(zabka_per_region_df['count_zabka'].max())
    display(zabka_per_region_df.head())
    
    # zabka_per_region_df.to_csv(f'../../data/downstream_tasks/zabka_shops/{city}_zabkas_in_hexes_res_{resolution}.csv', index=False)
    
    return zabka_per_region_df

In [6]:
zabka_data = pd.read_csv("../data/downstream_tasks/zabka_shops/zabka_shops.csv")
zabka_data


,id,slug,openTime,closeTime,city,address,postcode,voivodeship,county,community,region,salesRegion,openTimeSeconds,closeTimeSeconds,lat,lng,services
0,ID06093,"ID06093,gdansk-jabloniowa-29a",06:00,23:00,Gdańsk,Jabłoniowa 29A,80-175,Pomorskie,Gdańsk,GDAŃSK (GMINA MIEJSKA),DS3,PS3.6.3,21600.0,82800.0,54.330567,18.557187,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
1,ID03871,"ID03871,gorzow-wielkopolski-obroncow-pokoju-38...",06:00,23:00,Gorzów Wielkopolski,Obrońców Pokoju 38 nr 38 I,66-400,Lubuskie,Gorzów Wielkopolski,GORZÓW WIELKOPOLSKI (GMINA MIE,DS2,PS2.5.3,21600.0,82800.0,52.764806,15.264941,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
2,ID06169,"ID06169,ruda-slaska-ul-niedurnego-45-lok-1",06:00,23:00,Ruda Śląska,ul. Niedurnego 45 lok. 1,41-709,Śląskie,Ruda Śląska,RUDA ŚLĄSKA (GMINA MIEJSKA),DS8,PS8.2.2,21600.0,82800.0,50.284140,18.876000,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
3,ID06264,"ID06264,warszawa-ul-zeganska-18",06:00,23:00,Warszawa,ul. Żegańska 18,04-713,Mazowieckie,Warszawa,"WARSZAWA (GMINA MIEJSKA, MIAST",DS5,PS5.4.1,21600.0,82800.0,52.205620,21.172909,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
4,ID05100,"ID05100,wejherowo-dworcowa-2",06:00,23:00,Wejherowo,Dworcowa 2,84-200,Pomorskie,wejherowski,WEJHEROWO (GMINA MIEJSKA),DS3,PS3.2.3,21600.0,82800.0,54.603632,18.228519,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9754,ID0A221,"ID0A221,kedzierzyn-kozle-ul-gliwicka-1a",06:00,23:00,Kędzierzyn-Koźle,UL. GLIWICKA 1A,47-224,Opolskie,kędzierzyńsko-kozielski,KĘDZIERZYN-KOŹLE (GMINA MIEJSK,DS7,PS7.5.1,21600.0,82800.0,50.336754,18.183524,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9755,ID0B739,"ID0B739,lewin-brzeski-ul-kosciuszki-37d",06:00,23:00,Lewin Brzeski,UL. KOŚCIUSZKI 37D,49-340,Opolskie,brzeski (woj. opolskie),LEWIN BRZESKI (GMINA MIEJSKO-W,DS7,PS7.5.5,21600.0,82800.0,50.751824,17.615581,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9756,ID0C796,"ID0C796,uniejow-ul-targowa-18",00:00,23:59,Uniejów,UL. Targowa 18,99-210,Łódzkie,poddębicki,UNIEJÓW (GMINA MIEJSKO-WIEJSKA,DS6,PS6.6.5,0.0,86340.0,51.976362,18.791962,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9757,ID0D004,"ID0D004,warszawa-ul-zamkowa-8-lok-u1",06:00,23:00,Warszawa,UL. Zamkowa 8 lok. U1 .,03-890,Mazowieckie,Warszawa,"WARSZAWA (GMINA MIEJSKA, MIAST",DS5,PS5.4.1,21600.0,82800.0,52.274640,21.081131,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"


In [8]:
gdf = gpd.GeoDataFrame(
            zabka_data.drop(["lat", "lng"], axis=1),
            geometry=gpd.points_from_xy(x=zabka_data["lng"], y=zabka_data["lat"]),
            crs="EPSG:4326",
        )
gdf.head()

,id,slug,openTime,closeTime,city,address,postcode,voivodeship,county,community,region,salesRegion,openTimeSeconds,closeTimeSeconds,services,geometry
0,ID06093,"ID06093,gdansk-jabloniowa-29a",06:00,23:00,Gdańsk,Jabłoniowa 29A,80-175,Pomorskie,Gdańsk,GDAŃSK (GMINA MIEJSKA),DS3,PS3.6.3,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (18.55719 54.33057)
1,ID03871,"ID03871,gorzow-wielkopolski-obroncow-pokoju-38...",06:00,23:00,Gorzów Wielkopolski,Obrońców Pokoju 38 nr 38 I,66-400,Lubuskie,Gorzów Wielkopolski,GORZÓW WIELKOPOLSKI (GMINA MIE,DS2,PS2.5.3,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (15.26494 52.76481)
2,ID06169,"ID06169,ruda-slaska-ul-niedurnego-45-lok-1",06:00,23:00,Ruda Śląska,ul. Niedurnego 45 lok. 1,41-709,Śląskie,Ruda Śląska,RUDA ŚLĄSKA (GMINA MIEJSKA),DS8,PS8.2.2,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (18.87600 50.28414)
3,ID06264,"ID06264,warszawa-ul-zeganska-18",06:00,23:00,Warszawa,ul. Żegańska 18,04-713,Mazowieckie,Warszawa,"WARSZAWA (GMINA MIEJSKA, MIAST",DS5,PS5.4.1,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (21.17291 52.20562)
4,ID05100,"ID05100,wejherowo-dworcowa-2",06:00,23:00,Wejherowo,Dworcowa 2,84-200,Pomorskie,wejherowski,WEJHEROWO (GMINA MIEJSKA),DS3,PS3.2.3,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (18.22852 54.60363)


In [9]:
gdf_wro = gdf[gdf['city'] == 'Wrocław']
gdf_wro

,id,slug,openTime,closeTime,city,address,postcode,voivodeship,county,community,region,salesRegion,openTimeSeconds,closeTimeSeconds,services,geometry
28,ID01171,"ID01171,wroclaw-hlaski-26a",06:00,23:00,Wrocław,Hłaski 26a,54-608,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.2.3,21600.0,82800.0,"DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (16.94955 51.10132)
32,ID08794,"ID08794,wroclaw-uczniowska-1",06:00,23:00,Wrocław,Uczniowska 1,52-222,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.4.4,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.02829 51.06116)
33,ID02567,"ID02567,wroclaw-ul-krzywoustego-63a",06:00,23:00,Wrocław,ul. Krzywoustego 63a,51-165,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.3.6,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.07466 51.13782)
46,ID01669,"ID01669,wroclaw-sokola-45",06:00,23:00,Wrocław,Sokola 45,53-145,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.1.3,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.00675 51.08269)
49,ID05523,"ID05523,wroclaw-ul-gagarina-27-lok-7",06:00,23:00,Wrocław,ul. GAGARINA 27 lok. 7,54-620,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.2.3,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (16.93386 51.09495)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9454,ID02576,"ID02576,wroclaw-ul-legnicka-55d-u1",06:00,23:00,Wrocław,ul. Legnicka 55D/U1,54-203,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.2.2,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (16.99051 51.12353)
9499,ID06650,"ID06650,wroclaw-ul-rynek-8-lok-1a",06:00,23:00,Wrocław,ul. Rynek 8 Lok. 1A,50-106,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.1.1,21600.0,82800.0,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.03028 51.11007)
9566,ID01208,"ID01208,wroclaw-badury-4",06:00,23:00,Wrocław,Badury 4,51-507,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.3.4,21600.0,82800.0,"BIH,DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.14071 51.10802)
9572,ID05371,"ID05371,wroclaw-marca-polo-37a",06:00,23:00,Wrocław,Marca Polo 37a,51-504,Dolnośląskie,Wrocław,WROCŁAW (GMINA MIEJSKA),DS7,PS7.3.4,21600.0,82800.0,"BIH,DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC",POINT (17.11924 51.10957)


In [23]:
df = pd.DataFrame(columns=['h3_id', 'count_zabka', 'city_name'])
for City in ['Kraków', 'Poznań', 'Szczecin', 'Wrocław', 'Warszawa']:
    gdf_city = gdf[gdf['city'] == City]
    zprdf =zabka_per_region(City, gdf_city)
    zprdf['city_name'] = City
    df = pd.concat([df, zprdf], ignore_index=True)
    

5.0

,h3_id,count_zabka
0,891e05a6d93ffff,0.0
1,891e2e6b60bffff,0.0
2,891e2e692c3ffff,0.0
3,891e2e6e647ffff,0.0
4,891e2e6ae93ffff,0.0


5.0

,h3_id,count_zabka
0,891e24aac3bffff,1.0
1,891e24a8c33ffff,0.0
2,891e24a12d7ffff,0.0
3,891e24a165bffff,0.0
4,891e24b8017ffff,0.0


4.0

,h3_id,count_zabka
0,891f0e78273ffff,0.0
1,891f0e45a27ffff,0.0
2,891f0e6a307ffff,0.0
3,891f1db6917ffff,0.0
4,891f0e79a4bffff,0.0


5.0

,h3_id,count_zabka
0,891e2040c23ffff,1.0
1,891e204320bffff,0.0
2,891e204732bffff,0.0
3,891e2041987ffff,0.0
4,891e20422c7ffff,0.0


5.0

,h3_id,count_zabka
0,891f53533c7ffff,0.0
1,891f5234b7bffff,0.0
2,891f53d9a77ffff,0.0
3,891f53530b3ffff,0.0
4,891f53c9bcfffff,0.0


In [20]:
df

,region_id,count_zabka,city_name
0,891e05a6d93ffff,0.0,Kraków
1,891e2e6b60bffff,0.0,Kraków
2,891e2e692c3ffff,0.0,Kraków
3,891e2e6e647ffff,0.0,Kraków
4,891e2e6ae93ffff,0.0,Kraków
...,...,...,...
18634,891f535226bffff,0.0,Warszawa
18635,891f53c802fffff,0.0,Warszawa
18636,891f53c9067ffff,4.0,Warszawa
18637,891f53c9bb3ffff,0.0,Warszawa


In [24]:
df.to_parquet("../data/results_showcase/zabka_shops/zabka_counts.parquet")

In [29]:
from shapely import wkt
gg = pd.read_parquet("../data/results_showcase/zabka_shops/zabka_locations.parquet")
gg['geometry'] = gg['geometry'].apply(wkt.loads)
gdfx= gpd.GeoDataFrame(gg, geometry='geometry')
gdfx

,city_name,geometry
0,Kraków,POINT (19.91426 50.06739)
1,Kraków,POINT (19.90577 50.08373)
2,Kraków,POINT (20.01262 50.06982)
3,Kraków,POINT (20.00939 50.07318)
4,Kraków,POINT (19.98519 50.04042)
...,...,...
2278,Warszawa,POINT (21.05551 52.29839)
2279,Warszawa,POINT (21.05811 52.30265)
2280,Warszawa,POINT (20.99362 52.18255)
2281,Warszawa,POINT (21.04137 52.25028)


In [30]:
gdfx.to_parquet("../data/results_showcase/zabka_shops/zabka_locations.parquet")